# 04. Synthetic network analysis
## Project: Bicycle node network loop analysis

This notebook analyses synthetic networks (lattices).

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-10-28  
Last modified: 2024-10-28

## To do
- [ ] Bring into same data format as real networks
- [X] Calculate linklength vs face loops (just 3*, 4*, 6* ranges)
- [ ] Analyze: size/linklength vs loops, correspondence with scenarios
- [ ] Analyze: border effects

## Parameters

In [ ]:
%run -i setup_parameters.py
load_data = True  # Set to False if data are huge and have already been loaded
debug = True  # Set to True for extra plots and verbosity

size = 8
linklength = 10  # km

## Functions

In [ ]:
%run -i functions.py

## Load data

In [ ]:
if load_data:
    if LOOP_LENGTH_BOUND:
        llb_string = "_maxlength" + str(LOOP_LENGTH_BOUND)
    else:
        llb_string = ""

    with open(
        PATH["data_out"]
        + "loopcensus_"
        + str(LOOP_NUMNODE_BOUND)
        + llb_string
        + ".pkl",
        "rb",
    ) as f:
        allloops = pickle.load(f)
        alllooplengths = pickle.load(f)
        allloopnumnodes = pickle.load(f)
        allloopmaxslopes = pickle.load(f)
        Gnx = pickle.load(f)
        LOOP_NUMNODE_BOUND = pickle.load(f)
        nodes_id = pickle.load(f)
        nodes_coords = pickle.load(f)
        numloops = pickle.load(f)
        faceloops = pickle.load(f)

## Analytical comparison link lengths vs faceloops

In [ ]:
print("Target link length: " + str(LINK_LIMIT[0]) + " to " + str(LINK_LIMIT[1]))
print("")
print(
    "Target face loop length: "
    + str(FACELOOP_LIMIT[0])
    + " to "
    + str(FACELOOP_LIMIT[1])
)
for n in [3, 4, 6]:
    print(
        " "
        + str(n)
        + "-gon face loop length: "
        + str(n * LINK_LIMIT[0])
        + " to "
        + str(n * LINK_LIMIT[1])
    )

for n in [3, 4]:
    print(
        "S"
        + str(n)
        + "-gon face loop length: "
        + str(2 * n * LINK_LIMIT[0])
        + " to "
        + str(n * LINK_LIMIT[1])
    )

In [ ]:
degree_hist = nx.degree_histogram(Gnx)
degree_hist

## Create synthetic networks

In [ ]:
Gnx_tri = nx.triangular_lattice_graph(size, size)
# Add staggered triangle (all degree 3 nodes)

Gnx_grid = nx.grid_2d_graph(size, size)
pos = dict(zip(Gnx_grid, Gnx_grid))  # Source: https://stackoverflow.com/a/37106552
nx.set_node_attributes(Gnx_grid, pos, "pos")
# Add staggered grid (all degree 3 nodes)

Gnx_hex = nx.hexagonal_lattice_graph(size, size)

# Add hyperuniform(-like) points voronoi tesselation? (all degree 3 nodes)

Scale by linklength:

In [ ]:
nx.set_node_attributes(
    Gnx_tri,
    {
        k: (linklength * v[0], linklength * v[1])
        for (k, v) in nx.get_node_attributes(Gnx_tri, "pos").items()
    },
    "pos",
)
nx.set_node_attributes(
    Gnx_grid,
    {
        k: (linklength * v[0], linklength * v[1])
        for (k, v) in nx.get_node_attributes(Gnx_grid, "pos").items()
    },
    "pos",
)
nx.set_node_attributes(
    Gnx_hex,
    {
        k: (linklength * v[0], linklength * v[1])
        for (k, v) in nx.get_node_attributes(Gnx_hex, "pos").items()
    },
    "pos",
)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3))
nx.draw(Gnx_tri, pos=nx.get_node_attributes(Gnx_tri, "pos"), ax=axs[0], node_size=8)
nx.draw(Gnx_grid, pos=nx.get_node_attributes(Gnx_grid, "pos"), ax=axs[1], node_size=8)
nx.draw(Gnx_hex, pos=nx.get_node_attributes(Gnx_hex, "pos"), ax=axs[2], node_size=8)

To do: Make it momepy, make "pos" the geometry

In [ ]:
Gnx_tri_nodes, Gnx_tri_links = momepy.nx_to_gdf(net=Gnx_tri, points=True, lines=True)
Gnx_tri_nodes.plot()

In [ ]:
Gnx_tri_nodes.geometry.x

In [ ]:
Gnx_tri_nodes.pos

In [ ]:
for index, row in Gnx_tri_nodes.iterrows():
    print(row["pos"], row["geometry"])

In [ ]:
Gnx_tri_nodes.set_geometry("pos", drop=None, inplace=True)  # not working